In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r"C:\Users\pj\Desktop\ETL\DATA\Google Play Store Apps Dataset\googleplaystore_user_reviews.csv")
df.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


In [3]:
df.Sentiment.value_counts()

Positive    23998
Negative     8271
Neutral      5163
Name: Sentiment, dtype: int64

In [4]:
df.Sentiment.isna().sum()

26863

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64295 entries, 0 to 64294
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   App                     64295 non-null  object 
 1   Translated_Review       37427 non-null  object 
 2   Sentiment               37432 non-null  object 
 3   Sentiment_Polarity      37432 non-null  float64
 4   Sentiment_Subjectivity  37432 non-null  float64
dtypes: float64(2), object(3)
memory usage: 2.5+ MB


In [6]:
df = df.dropna(axis=0)
df.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000
5,10 Best Foods for You,Best way,Positive,1.00,0.300000


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37427 entries, 0 to 64230
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   App                     37427 non-null  object 
 1   Translated_Review       37427 non-null  object 
 2   Sentiment               37427 non-null  object 
 3   Sentiment_Polarity      37427 non-null  float64
 4   Sentiment_Subjectivity  37427 non-null  float64
dtypes: float64(2), object(3)
memory usage: 1.7+ MB


In [8]:
df.Sentiment.value_counts()

Positive    23998
Negative     8271
Neutral      5158
Name: Sentiment, dtype: int64

In [9]:
import re

replace_wo_space = re.compile("[.;:!\'?,\"()\[\]]")
replace_with_space = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

reviews = df['Translated_Review'].str.replace(replace_wo_space,'').str.lower()
reivews = reviews.str.replace(replace_with_space,'')
reviews.head(10)

0     i like eat delicious food thats im cooking foo...
1       this help eating healthy exercise regular basis
3            works great especially going grocery store
4                                          best idea us
5                                              best way
6                                               amazing
8                                   looking forward app
9                   it helpful site  it help foods get 
10                                             good you
11    useful information the amount spelling errors ...
Name: Translated_Review, dtype: object

In [10]:
df['Sentiment'] = df['Sentiment'].map({'Positive': 2, 'Neutral': 1, 'Negative':0})
df.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,2,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,2,0.25,0.288462
3,10 Best Foods for You,Works great especially going grocery store,2,0.40,0.875000
4,10 Best Foods for You,Best idea us,2,1.00,0.300000
5,10 Best Foods for You,Best way,2,1.00,0.300000


In [11]:
reviews.count()

37427

In [12]:
df.Sentiment.count()

37427

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(binary=True)
cv.fit(reviews)
X = cv.transform(reviews)

print(X.shape)

(37427, 23545)


In [14]:
from sklearn.model_selection import train_test_split

target = df['Sentiment']
#target = pd.get_dummies(df['Sentiment'])

X_train, X_test, y_train, y_test = train_test_split(X, target, test_size = 0.3, random_state=42)

In [15]:
from sklearn.linear_model import LogisticRegressionCV

grid_params = [0.25, 0.5, 0.75, 1]
lr = LogisticRegressionCV(Cs=grid_params, solver='newton-cg', cv=5, multi_class='multinomial', random_state=42)
lr.fit(X_train, y_train)

LogisticRegressionCV(Cs=[0.25, 0.5, 0.75, 1], cv=5, multi_class='multinomial',
                     random_state=42, solver='newton-cg')

In [16]:
lr.scores_

{0: array([[0.91221374, 0.91698473, 0.92041985, 0.92385496],
        [0.90973282, 0.91545802, 0.91793893, 0.91908397],
        [0.91240458, 0.91717557, 0.91965649, 0.91965649],
        [0.92174079, 0.92498568, 0.92708532, 0.92765795],
        [0.91506013, 0.91715976, 0.92021378, 0.92154991]]),
 1: array([[0.91221374, 0.91698473, 0.92041985, 0.92385496],
        [0.90973282, 0.91545802, 0.91793893, 0.91908397],
        [0.91240458, 0.91717557, 0.91965649, 0.91965649],
        [0.92174079, 0.92498568, 0.92708532, 0.92765795],
        [0.91506013, 0.91715976, 0.92021378, 0.92154991]]),
 2: array([[0.91221374, 0.91698473, 0.92041985, 0.92385496],
        [0.90973282, 0.91545802, 0.91793893, 0.91908397],
        [0.91240458, 0.91717557, 0.91965649, 0.91965649],
        [0.92174079, 0.92498568, 0.92708532, 0.92765795],
        [0.91506013, 0.91715976, 0.92021378, 0.92154991]])}

In [17]:
lr.C_

array([1., 1., 1.])

In [18]:
from sklearn.metrics import accuracy_score

y_train_pred = lr.predict(X_train)
print("Training accuracy is {}".format(accuracy_score(y_train, y_train_pred)))

y_test_pred = lr.predict(X_test)
print("Testing accuracy is {}".format(accuracy_score(y_test, y_test_pred)))

Training accuracy is 0.9856095885182075
Testing accuracy is 0.9283996794015495


In [19]:
s = pd.Series(['this is a terrible app','i love this app', 'more to be expected'])
s = cv.transform(s)
lr.predict(s)

array([0, 2, 2], dtype=int64)

In [20]:
import nltk
nltk.download()
nltk.download('stopwords')

from nltk.corpus import stopwords

set(stopwords.words("english"))

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [21]:
from nltk.corpus import stopwords

heshe = stopwords.words('english')
reviews_stopworded = reviews.apply(lambda x: ' '.join([word for word in x.split()
                                                          if word not in heshe]))
reviews_stopworded.head()

0    like eat delicious food thats im cooking food ...
1           help eating healthy exercise regular basis
3           works great especially going grocery store
4                                         best idea us
5                                             best way
Name: Translated_Review, dtype: object

In [22]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pj\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [23]:
from nltk.stem import WordNetLemmatizer

lem = WordNetLemmatizer()
reviews_lem = reviews_stopworded.apply(lambda x: ' '.join([lem.lemmatize(word) for word in x.split()]))
reviews_lem.head()

0    like eat delicious food thats im cooking food ...
1           help eating healthy exercise regular basis
3            work great especially going grocery store
4                                          best idea u
5                                             best way
Name: Translated_Review, dtype: object

In [24]:
ngram_cv = CountVectorizer(binary=True, ngram_range=(1,2))
ngram_cv.fit(reviews_lem)
X = ngram_cv.transform(reviews_lem)

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.3, random_state=42)

# grid_params = [0.25, 0.5, 0.75, 1]
# lr_nltk = LogisticRegressionCV(Cs=grid_params, solver='newton-cg', cv=5, multi_class='multinomial', random_state=42)
# lr_nltk.fit(X_train, y_train)

from sklearn.linear_model import LogisticRegression

lr_nltk = LogisticRegression(solver='newton-cg', multi_class='multinomial', C=1.0, random_state=42)
lr_nltk.fit(X_train, y_train)

LogisticRegression(multi_class='multinomial', random_state=42,
                   solver='newton-cg')

In [26]:
y_train_pred = lr_nltk.predict(X_train)
print("Training accuracy is {}".format(accuracy_score(y_train, y_train_pred)))

y_test_pred = lr_nltk.predict(X_test)
print("Testing accuracy is {}".format(accuracy_score(y_test, y_test_pred)))

Training accuracy is 0.9945797389113673
Testing accuracy is 0.919405111764182


In [27]:
s = pd.Series(['this is a terrible app', 'i love this app', 'more to be expected'])
s = ngram_cv.transform(s)
lr_nltk.predict(s)

array([0, 2, 1], dtype=int64)

In [28]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV

# I had to set the max_iter because it was not able to converge the SVM in the default 1,000 iterations

grid_params = {'C':[0.25,0.5,0.75,1.0]}
svm = GridSearchCV(LinearSVC(multi_class='ovr', max_iter=2500, random_state=42),grid_params,cv=5)
svm.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=LinearSVC(max_iter=2500, random_state=42),
             param_grid={'C': [0.25, 0.5, 0.75, 1.0]})

In [29]:
svm.cv_results_

{'mean_fit_time': array([3.58384109, 5.79999247, 7.75744286, 9.86319494]),
 'std_fit_time': array([0.27140837, 0.31537352, 0.19651592, 0.61739975]),
 'mean_score_time': array([0.00408769, 0.00130553, 0.01225014, 0.        ]),
 'std_score_time': array([0.00527754, 0.00178235, 0.00615816, 0.        ]),
 'param_C': masked_array(data=[0.25, 0.5, 0.75, 1.0],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 0.25}, {'C': 0.5}, {'C': 0.75}, {'C': 1.0}],
 'split0_test_score': array([0.9120229 , 0.91030534, 0.90916031, 0.90801527]),
 'split1_test_score': array([0.91183206, 0.91221374, 0.91183206, 0.91183206]),
 'split2_test_score': array([0.91335878, 0.9139313 , 0.91354962, 0.91278626]),
 'split3_test_score': array([0.91735064, 0.91506013, 0.91601451, 0.91677801]),
 'split4_test_score': array([0.91715976, 0.91830502, 0.91715976, 0.91467837]),
 'mean_test_score': array([0.91434483, 0.91396311, 0.91354325, 0.91281799]),
 'std_te

In [30]:
svm.best_score_

0.9143448286413263

In [31]:
svm.best_estimator_

LinearSVC(C=0.25, max_iter=2500, random_state=42)

In [32]:
y_train_pred = svm.predict(X_train)
print("Training accuracy is {}".format(accuracy_score(y_train, y_train_pred)))

y_test_pred = svm.predict(X_test)
print("Testing accuracy is {}".format(accuracy_score(y_test, y_test_pred)))

Training accuracy is 0.996450110695473
Testing accuracy is 0.9218096001424883


In [33]:
s = pd.Series(['this is a terrible app', 'i love this app', 'more to be expected'])
s = ngram_cv.transform(s)
svm.predict(s)

array([0, 2, 1], dtype=int64)